# Instationary Parabolic Equations

In this tutorial we extend the elliptic problem from tutorial 01 to the
time dependent case. Following the method of lines a general approach to time-dependent
problems is presented.
This tutorial depends on tutorial 01.

# PDE Problem

In this tutorial we consider the following problem:
\begin{align}\label{equ:problem}
\partial_t u -\Delta u + q(u) &= f &&\text{in $\Omega\times\Sigma$},\\
u &= g &&\text{on $\Gamma_D\subseteq\partial\Omega$},\\
-\nabla u\cdot \nu &= j &&\text{on $\Gamma_N=\partial\Omega\setminus\Gamma_D$},\\
u &= u_0 &&\text{at $t=0$}. \label{equ:problemEnd}
\end{align}
This problem is a straightforward extension of the stationary
problem solved in tutorial 01.
The parameter functions $f$, $g$, $j$ may now also depend on time
and (with some restrictions) the subdivision into Dirichlet and Neumann boundary
can be time-dependent as well. The initial
condition $u_0$ is a function of $x\in\Omega$.

Multiplying with a test function and integrating in space
results in the following weak formulation [[8]](#ern):
Find $u\in L_2(t_0,t_0+T;u_g+V(t))$:
\begin{equation}
\frac{d}{dt} \int_\Omega u v \,dx+ \int_\Omega \nabla u \cdot \nabla v
+ q(u) v - f v \, dx + \int_{\Gamma_N} jv \, ds = 0 \qquad
\begin{array}{l}
\forall v \in V(t),\\
t \in \Sigma,
\end{array}
\label{eq:WeakForm}
\end{equation}
where $V(t) = \{v\in H^1(\Omega)\,:\, \text{$v=0$ on $\Gamma_D(t)$}\}$
and $H^1(\Omega)\ni u_g(t)|_{\Gamma_D}=g$. This can be written in a more compact way
with residual forms as follows:
\begin{equation*}
\frac{d}{dt} m^{\text{L2}}(u,v) + r^{\text{NLP}}(u,v) = 0 \quad \forall v \in V(t), t \in \Sigma.
\end{equation*}
where
\begin{equation*}
m^{\text{L2}}(u,v) = \int_\Omega u v \,dx
\end{equation*}
is the new temporal residual form, actually the $L_2$ inner product, and
\begin{equation*}
r^{\text{NLP}}(u,v) = \int_\Omega \nabla u \cdot \nabla v + (q(u)-f)v\,dx + \int_{\Gamma_N} jv\,ds ,
\end{equation*}
is the spatial residual form known from tutorial 01.
Under suitable assumptions it can be shown that problem \eqref{eq:WeakForm} has
a unique solution, [[8]](#ern) for the linear case.

# Finite Element Method

In order to arrive at a fully discrete formulation we follow the method of lines
paradigm:

1. Choose a finite-dimensional test space $V_h(t)\subset V(t)$. Then \eqref{eq:WeakForm} results in a system of ordinary differential equations for the coefficients $z_j(t)$ in the expansion of $u_h(t)=\sum_{j=1}^n (z_j(t))_j \phi_j$.
2. Choose an appropriate method to integrate the system of ordinary differential equations (ODEs).

The finite-dimensional space we choose here is just the conforming
finite element space $V_h^{k,d}(\mathcal{T}_h,t)$ introduced in tutorial 01.
It may now depend also on time due to the time-dependent splitting of the boundary
in Dirichlet and Neumann part. Also the function $u_{h,g}(t)$ depends on time
and we have $u_h(t)\in U_h(t) = u_{h,g}(t) + V_h(t)$.

For the integration of the system of ODEs, subdivide the time interval into
not necessarily equidistant subintervals:
\begin{equation*}
\overline{\Sigma} = \{t^{0}\} \cup (t^0,t^1] \cup \ldots \cup (t^{N-1},t^N]
\end{equation*}
with $t^0=t_0$, $t^N=t_0+T$, $t^{k-1}<t^k$ for $1\leq k\leq N$ and
set  the time step to $\Delta t^k=t^{k+1}-t^k$.

One of the simplest ODE integrators is
the one-step-$\theta$ rule which reads:
\begin{equation}
\label{Eq:InstationaryProblem}
\begin{split}
&\text{Find $u_h^{k+1}\in U_h(t^{k+1})$ s.t.:}
\ \frac{1}{\Delta t_k}(m_h^\text{L2}(u_h^{k+1},v;t^{k+1})-m_h^\text{L2}(u_h^{k},v;t^{k})) + \\
&\hspace{25mm}\theta r_h^\text{NLP}(u_h^{k+1},v;t^{k+1}) + (1-\theta) r_h^\text{NLP}(u_h^{k},v;t^{k}) = 0
\quad \forall v\in V_h(t^{k+1}).
\end{split}
\end{equation}

Reordering terms shows that this method results in the solution
of a nonlinear system per time step which has the same structure as before:
\begin{equation*}
\text{Find $u_h^{k+1}\in U_h(t^{k+1})$ s.t.:}
\quad r_h^{\theta,k} (u_h^{k+1},v) + s_h^{\theta,k}(v) = 0
\quad \forall v\in V_h(t^{k+1}).
\end{equation*}
where
\begin{align*}
r^{\theta,k}_h(u,v) &= m_h^\text{L2}(u,v;t^{k+1})+\Delta t^k \theta r_h^\text{NLP}(u,v;t^{k+1}) ,\\
s^{\theta,k}_h(v) &= -m_h^\text{L2}(u^k_h,v;t^k) + \Delta t^k (1-\theta) r_h^\text{NLP}(u_h^k,v;t^k) .
\end{align*}
Implementation-wise the new residual form is comprised of a linear combination
of temporal and spatial residual forms.

The one step $\theta$ method results in the implicit Euler method for $\theta=1$,
the Crank-Nicolson method for $\theta=1/2$ and the explicit Euler method for $\theta=0$.
A large number of alternative methods are possible. In particular the class of
Runge-Kutta methods is introduced below.

## Runge-Kutta Methods in Shu-Osher Form

For the temporal and spatial residual forms $m_h$ and $r_h$, Runge-Kutta
methods can be written in the Shu-Osher form ([[9]](#ShuOsher88),[[10]](#gottlieb11)):


1. $u_h^{(0)} = u_h^{k}$.
2. For $i=1,\ldots,s\in\mathbb{N}$, find $u_h^{(i)}\in u_{h,g}(t^k+d_i \Delta t^k)
    + V_h(t^{k+1})$:
\begin{equation*}
\begin{split}
\sum\limits_{j=0}^{s} \biggl[a_{ij} m_h&\left(u_h^{(j)},v;t^k+d_j \Delta t^k\right) \\
&\qquad + b_{ij} \Delta t^k r_h\left(u_h^{(j)}, v;t^k+d_j \Delta t^k\right) \biggr] = 0
\qquad \forall v\in V_h(t^{k+1}).
\end{split}
\end{equation*}
3. $u_h^{k+1} = u_h^{(s)}$.

Here we assume that the same type of boundary condition
holds through the interval $(t^k,t^{k+1}]$. The parameter $s$ denotes
the number of stages of the scheme.
An $s$-stage scheme is given by the parameters
\begin{equation*}
A = \left[\begin{array}{ccc}
a_{10} & \ldots & a_{1s}\\
\vdots &  & \vdots\\
a_{s0} & \ldots & a_{ss}
\end{array}\right],
\quad B = \left[\begin{array}{ccc}
b_{10} & \ldots & b_{1s}\\
\vdots &  & \vdots\\
b_{s0} & \ldots & b_{ss}
\end{array}\right],
\quad d = \left(
d_{0}, \ldots, d_{s}
\right)^T.
\end{equation*}
*Explicit* schemes are characterized by $a_{ij} = 0$ for $j>i$ and $b_{ij}=0$ for $j\geq i$.
*Diagonally implicit* schemes are characterized by $a_{ij} = b_{ij}= 0$ for $j>i$.
Fully implicit schemes where $A$ and $B$ are full are not considered in PDELab.
Without loss of generality it can be assumed that $a_{ii}=1$. Moreover,
some diagonally implicit schemes, in particular all those implemented in PDELab right
now, satisfy $b_{ii}=b\,\forall i$. This means that in case of a linear problem the matrix is the
same in all steps and needs to be assembled only once.
PDELab allows you to add new schemes by specifying $A$, $B$ and $d$.
All explicit Runge-Kutta methods and most implicit Runge-Kutta methods
can be brought to the Shu-Osher form.
Some examples are:


 - One step $\theta$ scheme (introduced above):
\begin{equation*}
A = \left[\begin{array}{cc}
-1 & 1
\end{array}\right],
\quad B = \left[\begin{array}{cc}
1-\theta & \theta
\end{array}\right],
\quad d = \left(
0, 1
\right)^T.
\end{equation*}
Explicit/implicit Euler ($\theta\in\{0,1\}$), Crank-Nicolson ($\theta=1/2$).
 -  Heun's second order explicit method
\begin{equation*}
A = \left[\begin{array}{ccc}
-1 & 1 & 0\\
-1/2 & -1/2 & 1\\
\end{array}\right],
\quad B = \left[\begin{array}{ccc}
1 & 0 & 0\\
0 & 1/2 & 0\\
\end{array}\right],
\quad d = \left(
0, 1, 1
\right)^T.
\end{equation*}
 -  Alexander's two-stage second order strongly S-stable method \cite{alexander:77}:
\begin{equation*}
A = \left[\begin{array}{ccc}
-1 & 1 & 0\\
-1 & 0 & 1\\
\end{array}\right],
\quad B = \left[\begin{array}{ccc}
0 & \alpha     & 0\\
0 & 1-\alpha & \alpha\\
\end{array}\right],
\quad d = \left(
0, \alpha, 1
\right)^T
\end{equation*}
with $\alpha=1-\sqrt{2}/{2}$.
 -  Fractional step $\theta$ \cite{Meidner201545}, three stage second order strongly A-stable scheme:
\begin{equation*}
A = \left[\begin{array}{rrrr}
-1 & 1 & 0 & 0\\
0  & -1 & 1 & 0\\
0  & 0 & -1 & 1\\
\end{array}\right],
\quad B = \left[\begin{array}{rrrr}
\theta (1-\alpha) & \theta\alpha & 0 & 0\\
0 & \theta'\alpha & \theta' (1-\alpha) & 0\\
0 & 0 & \theta(1-\alpha) & \theta\alpha
\end{array}\right],
\quad d = \left(
0, \theta, 1-\theta, 1
\right)^T
\end{equation*}
with $\theta=1-\sqrt{2}/{2}$, $\alpha=2\theta$, $\theta' = 1-2\theta = 1-\alpha = \sqrt{2}-1$.
Note also that $\theta\alpha= \theta' (1-\alpha) = 2\theta^2$.


## Explicit Time Stepping Schemes

Considering the case of the explicit Euler method ($\theta=0$) in \eqref{Eq:InstationaryProblem}
results in the problem: Find $u_h^{k+1}\in U_h(t^{k+1})$ s.t.:
\begin{equation*}
 m_h^\text{L2}(u_h^{k+1},v;t)-m_h^\text{L2}(u_h^{k},v;t) +
\Delta t^k r_h^\text{NLP}(u_h^{k},v;t) = 0
\quad \forall v\in V_h(t^{k+1}).
\end{equation*}
For certain spatial schemes, e.g. finite volume or discontinuous Galerkin,
and exploiting that $m_h^\text{L2}$ is bilinear, the corresponding algebraic system
to be solved is (block-) diagonal:
\begin{equation}
Dz^{k+1} = s^k - \Delta t^k q^k.
\end{equation}
Moreover, a stability condition restricting the time step $\Delta t^k$
has to be obeyed. The maximum allowable time step can be computed
explicitly for the simplest schemes and depends on the mesh $\mathcal{T}_h$.
For explicit time-stepping schemes therefore the following algorithm is employed:
1. While traversing the mesh assemble the vectors $s^k$ and
$q^k$ separately and compute the maximum time step $\Delta t^k$.
2. Form the right hand side $b^k=s^k - \Delta t^k q^k$ and ``solve'' the
diagonal system $Dz^{k+1} = b^k$ (can be done in one step).

This procedure can be applied also to more general time-stepping schemes
such as strong stability preserving Runge-Kutta methods [[16]](#Shu).

# Realization in PDELab

## Parameter Class in `problem.hh`

The parameter class provides all data of the PDE problem: Coefficient functions,
boundary and initial conditions. They all may depend on time now in comparison to tutorial 01.
In order to pass the time argument there are at least two options:

1. Extend the interface of all methods by an additional time
argument.
2. Pass the evaluation time via a separate function and store the
time in a private data member for subsequent spatial evaluations.

The second approach is taken in PDELab as it allows the reuse
of classes from the stationary case. Interfaces of methods are not changed
and only an additional method needs to be provided.
This new method on the class `Problem` has the following implementation:
```c++ 
//! Set time in instationary case
void setTime (Number t_)
{
  t = t_;
}
```
The method just stores the given time in an internal data member.

The use of the time is shown by the Dirichlet boundary condition
extension method implementing the function
$u_g(x,t) = \sin(2\pi t)*\prod_{i=1}^{d-1} \sin^2(\pi x_i)\sin^2(10\pi x_i)$:

```c++
template<typename E, typename X>
Number g (const E& e, const X& x) const
{
  auto global = e.geometry().global(x);
  Number s=sin(2.0*M_PI*t);
  for (std::size_t i=1; i<global.size(); i++)
    s*=sin(global[i]*M_PI)*sin(global[i]*M_PI);
  for (std::size_t i=1; i<global.size(); i++)
    s*=sin(10*global[i]*M_PI)*sin(10*global[i]*M_PI);
  return s;
 }
```
Note that there is no extra method for the initial condition. The method `g` is assumed to provide the initial condition for the initial time.

## Instantiating grid

In [ ]:
#include <dune/jupyter.hh>
#include "nonlinearheatfem.hh"
#include "problem.hh"
#include <sys/stat.h>

In [ ]:
const int dim = 2;
const int degree = 1;
using RF = double; 

In [ ]:
Dune::ParameterTree ptree;
Dune::ParameterTreeParser ptreeparser;
ptreeparser.readINITree("tutorial03.ini",ptree);

// read ini file
const int refinement = ptree.get<int>("grid.refinement");

In [ ]:
//eine Mglk
/*typedef Dune::YaspGrid<dim> Grid;
typedef Grid::ctype DF;
Dune::FieldVector<DF,dim> L;
L[0] = ptree.get("grid.structured.LX",(double)1.0);
L[1] = ptree.get("grid.structured.LY",(double)1.0);
std::array<int,dim> N;
N[0] = ptree.get("grid.structured.NX",(int)10);
N[1] = ptree.get("grid.structured.NY",(int)10);
std::shared_ptr<Grid> gridp = std::shared_ptr<Grid>(new Grid(L,N));
*/

In [ ]:
//using grid factory
using Grid = Dune::YaspGrid<dim>;
using DF = Grid::ctype ;
// define the extensions of the domain
Dune::FieldVector<double,dim> lowerleft(0.0);
Dune::FieldVector<double,dim> upperright;
upperright[0] = ptree.get("grid.structured.LX",(double)1.0);
upperright[1] = ptree.get("grid.structured.LZ",(double)1.0);

std::array< unsigned int, dim> N;
N[0] = ptree.get("grid.structured.NX",(int)10);
N[1] = ptree.get("grid.structured.NY",(int)10);

// build a structured grid
auto grid = Dune::StructuredGridFactory<Grid>::createCubeGrid(lowerleft, upperright, N);

In [ ]:
grid->globalRefine(refinement);
using GV = Grid::LeafGridView ;
GV gv=grid->leafGridView();
using FEM = Dune::PDELab::QkLocalFiniteElementMap<GV,DF,double,degree>;
FEM fem(gv);

## Changes due to Instationarity

We now go through the changes which are due to instationarity. The first change concerns the initialization of simulation time and set-up of the parameter object:

The first change concerns the initialization of simulation time and set-up of the parameter object:

In [ ]:
// make user functions and set initial time
RF time = 0.0;
RF eta = ptree.get("problem.eta",(RF)1.0);
Problem<RF> problem(eta);
problem.setTime(time);

Now a PDELab grid function can be constructed:

In [ ]:
auto g = Dune::PDELab::
makeInstationaryGridFunctionFromCallable(
    gv,
    [&](const auto& e, const auto& x){
        return problem.g(e,x);},
    problem);;

A new function `makeInstationaryGridFunctionFromCallable` is provided since the grid function produced by it now also provides a method `setTime` to pass the time argument.
Note that the lambda closure and the parameter object need to be
provided as the `setTime` method is only on the parameter class
and not on the lambda closure.

In [ ]:
auto b = Dune::PDELab::
makeBoundaryConditionFromCallable(
    gv,
    [&](const auto& i, const auto& x){
        return problem.b(i,x);}
);;

Setting up the grid function space and interpolating the initial condition is the same as before. 

In [ ]:
// Make grid function space
using CON = Dune::PDELab::ConformingDirichletConstraints;
using VBE = Dune::PDELab::ISTL::VectorBackend<>;
using GFS = Dune::PDELab::GridFunctionSpace<GV,FEM,CON,VBE>;
GFS gfs(gv,fem);
gfs.name("Vh");

// Assemble constraints
using CC = typename GFS::template
ConstraintsContainer<RF>::Type;
CC cc;
Dune::PDELab::constraints(b,gfs,cc); // assemble constraints
std::cout << "constrained dofs=" << cc.size() << " of "
        << gfs.globalSize() << std::endl;

// A coefficient vector
using Z = Dune::PDELab::Backend::Vector<GFS,RF>;
Z z(gfs); // initial value

// Make a grid function out of it
using ZDGF = Dune::PDELab::DiscreteGridFunction<GFS,Z>;
ZDGF zdgf(gfs,z);

// initialize simulation time, the coefficient vector
Dune::PDELab::interpolate(g,gfs,z);

Then the grid operator for the spatial part can be set up as in the stationary problem tutorial 01. We just use the new local operator `NonlinearHeatFEM`:

In [ ]:
// Make instationary grid operator
using LOP = NonlinearHeatFEM<Problem<RF>,FEM>;
LOP lop(problem);
using MBE = Dune::PDELab::ISTL::BCRSMatrixBackend<>;
int degree = ptree.get("fem.degree",(int)1);
MBE mbe((int)pow(1+2*degree,dim));
using GO0 = Dune::PDELab::GridOperator<GFS,GFS,LOP,MBE,
                                 RF,RF,RF,CC,CC>;
GO0 go0(gfs,cc,gfs,cc,lop,mbe);

The temporal part is done similarly and just employs another local operator
`L2`:

In [ ]:
using TLOP = L2<FEM>;
TLOP tlop;
using GO1 = Dune::PDELab::GridOperator<GFS,GFS,TLOP,MBE,
                                 RF,RF,RF,CC,CC>;
GO1 go1(gfs,cc,gfs,cc,tlop,mbe);

Spatial and temporal part are now combined using the new class `OneStepGridOperator` which can assemble the linear combinations needed in the Runge-Kutta methods:

In [ ]:
using IGO = Dune::PDELab::OneStepGridOperator<GO0,GO1>;
IGO igo(go0,go1);

Linear solver backend and Newton method can be set up in the same way as before, just using the `OneStepGridOperator`.

In [ ]:
// Select a linear solver backend
using LS = Dune::PDELab::ISTLBackend_SEQ_CG_AMG_SSOR<IGO>;
LS ls(100,0);

// solve nonlinear problem
using PDESOLVER = Dune::PDELab::Newton<IGO,LS,Z>;
PDESOLVER pdesolver(igo,z,ls);
pdesolver.setReassembleThreshold(0.0);
pdesolver.setVerbosityLevel(2);
pdesolver.setReduction(1e-8);
pdesolver.setMinLinearReduction(1e-4);
pdesolver.setMaxIterations(25);
pdesolver.setLineSearchMaxIterations(10);

Then a time-stepping scheme in the form of the matrices $A$, $B$ and the vector $d$ needs to be selected.
Several classes are already provided and it is very easy to write your own class:

In [ ]:
Dune::PDELab::OneStepThetaParameter<RF> method1(1.0);
Dune::PDELab::Alexander2Parameter<RF> method2;
Dune::PDELab::Alexander3Parameter<RF> method3;

The selection mechanism for the time-stepping scheme is implemented by setting a pointer to an appropriate method:

In [ ]:
int torder = ptree.get("fem.torder",(int)1);
Dune::PDELab::TimeSteppingParameterInterface<RF>*
  pmethod=&method1;
if (torder==1) pmethod = &method1;
if (torder==2) pmethod = &method2;
if (torder==3) pmethod = &method3;
if (torder<1||torder>3)
  std::cout<<"torder not in [1,3]"<<std::endl;

Note that`TimeSteppingParameterInterface` is the interface from which all time stepping parameter classes derive. 

Then an object of class`OneStepMethod` can be instantiated. This class is able to compute a single time step of a general Runge-Kutta one step method.

In [ ]:
using OSM = Dune::PDELab::OneStepMethod<RF,IGO,PDESOLVER,Z,Z>;
OSM  osm(*pmethod,igo,pdesolver);
osm.setVerbosityLevel(2);

Before entering the time loop we set up VTK output for the instationary case.
Paraview has several ways to handle sequences of output files. Here we use
an advanced method that writes a special \lstinline{pvd}-file providing the
names of the files for each individual step and the associated absolute time. This
is particularly useful when non-equidistant time steps are used. All the
data files for the time steps are put in a subdirectory in order not to clog your file system.
The following code creates a directory (the name is read from the ini-file):

In [ ]:
// prepare VTK writer and write first file
std::string filename=ptree.get("output.filename","output");
struct stat st;
if( stat( filename.c_str(), &st ) != 0 )
{
  int stat = 0;
  stat = mkdir(filename.c_str(),S_IRWXU|S_IRWXG|S_IRWXO);
  if( stat != 0 && stat != -1)
    std::cout << "Error: Cannot create directory "
              << filename << std::endl;
}

Now a `SubsamplingVTKWriter` and a `VTKSequenceWriter` can be instantiated:

In [ ]:
int subsampling=ptree.get("output.subsampling",(int)1);
using VTKWRITER = Dune::SubsamplingVTKWriter<GV>;
VTKWRITER vtkwriter(gv,Dune::refinementIntervals(subsampling));
using VTKSEQUENCEWRITER = Dune::VTKSequenceWriter<GV>;
VTKSEQUENCEWRITER vtkSequenceWriter(
  std::make_shared<VTKWRITER>(vtkwriter),filename,filename,"");

filled with data

In [ ]:
using VTKF = Dune::PDELab::VTKGridFunctionAdapter<ZDGF>;
vtkSequenceWriter.addVertexData(std::shared_ptr<VTKF>(
                           new VTKF(zdgf,"solution")));

and the first file is written

In [ ]:
vtkSequenceWriter.write(time,Dune::VTK::appendedraw);

Now the time loop is under user control:

In [ ]:
RF T = ptree.get("problem.T",(RF)1.0);
RF dt = ptree.get("fem.dt",(RF)0.1);
while (time<T-1e-8)
  {
    // assemble constraints for new time step
    problem.setTime(time+dt);
    Dune::PDELab::constraints(b,gfs,cc);

    // do time step
    Z znew(z);
    osm.apply(time,dt,z,g,znew);

    // accept time step
    z = znew;
    time+=dt;

    // output to VTK file
    vtkSequenceWriter.write(time,Dune::VTK::appendedraw);
  }

In [ ]:
vtkSequenceWriter

First the final time and time step are read from the parameter file. Then, for each time step the constraints are reassembled. Note that the boundary condition type is not supposed to change *within* a time step, so all stages of the Runge-Kutta Method
will use the same constraints (but not necessarily the same values at the
Dirichlet boundary).

Then, the `apply` method of the one step method is used to advance one step in time. Its arguments are the current `time`, the size of the time step `dt`, the coefficient vector `z` of the oldtime step, the boundary condition function `g` used to interpolate
boundary conditions for the stages and, as last argument, the new coefficient
vector`znew` to be computed as a result.

In order to advance to the next time step the coefficient vector is copied and the
time variable is advanced. Finally, the output file for the time step is written.

Intentionally, the time loop is under user control to allow other things to be done,
e.g. choose adaptive time step size, proceeding to important absolute times,
writing only  every $n$th output file and so on.

## Spatial Local Operator

The file `nonlinearheatfem.hh` contains the two local operators
for the spatial and the temporal part. The class `NonlinearHeatFEM` implements the
spatial part. It is actually very easy since the main part can be reused
from the `NonlinearPoissonFEM` local operator  implemented in tutorial 01 through
public inheritance:

```c++
template<typename Param, typename FEM>
class NonlinearHeatFEM :
  public NonlinearPoissonFEM<Param,FEM>,
  public Dune::PDELab::
    InstationaryLocalOperatorDefaultMethods<double>
{
  Param& param;
public:
  //! Pass on constructor
  NonlinearHeatFEM (Param& param_, int incrementorder_=0)
    : NonlinearPoissonFEM<Param,FEM>(param_,incrementorder_),
    param(param_)
  {}
  //! set time for subsequent evaluation
  void setTime (double t) {
    param.setTime(t);
  }
};
```

Just a few methods need to be added to the local operator.
Default implementations can be inherited from `InstationaryLocalOperatorDefaultMethods`. Here only the method `setTime` needs to be implemented to pass on the time to the parameter class for subsequent evaluations.

The following methods need to be provided by a local operator for instationary computations:

```c++
void setTime (R t_)
```  
to set the time for all subsequent method calls.

```c++
R getTime () const
```
to get the time set previously.

```c++
void preStep (RealType time, RealType dt, int stages)
```
This method is called at the beginning of a time step, before all stages.

```c++
void postStep ()
```
This method is called at the end of a time step, after all stages.

```c++
void preStage (RealType time, int r)
```
This method is called at the beginning of a Runge-Kutta stage, where $r$ is the number of the stage (starting with $r=1$).

```c++
int getStage () const
```
Return the number of the current stage.

```c++
void postStage ()
```
Called after the intermediate result of a stage has been computed.

```c++
RealType suggestTimestep (RealType dt) const
```
In an explicit scheme this method can be called at the end of stage 1 to suggest a stable time step for the explicit scheme.

## Temporal Local Operator

The temporal local operator needs to implement the residual form related to the $L_2$-inner product. It could also be used to compute an $L_2$ projection of some function.

Like in the spatial local operator we provide a finite element map as template parameter and derive from some helpful classes to provide default implementations of some methods:
```c++
template<typename FEM>
class L2
  : public Dune::PDELab::FullVolumePattern,
    public Dune::PDELab::LocalOperatorDefaultFlags,
    public Dune::PDELab::
  InstationaryLocalOperatorDefaultMethods<double>
```

The following flag indicates that we provide a `pattern_volume` method
specifying the sparsity pattern. Actually, this method is provided from the base class
`FullVolumePatter`:    
```c++    
enum { doPatternVolume = true };
```

The next flag indicates that just volume integrals depending on trial and test functions are required:
```c++  
enum { doAlphaVolume = true };
```

The implementation of `alpha_volume` is very similar to the reaction term in tutorial 01. For efficiency reasons also `jacobian_volume` is provided as well as the `*_apply_*` variants for matrix-free operation (which might be very useful here since the mass matrix is well conditioned).    

# Exercise

The code of \lstinline!tutorial03! solves the problem described in (\ref{equ:problem}) - (\ref{equ:problemEnd}), with the following choices applied:

  \begin{align}
    \label{ch1:first}
    q(u) &= \eta u^2 \\
    f &= 0 \\
    \Gamma_D &= \{x \in\partial\Omega \mid x_0 = 0 \} \\
    g(x,t) &= \sin(2\pi t) \prod_{i=1}^{d-1} \sin(\pi x_i)^2
    \sin(10\pi x_i)^2\\
    j(x,t) &= 0 \\
    u_0(x) &= g(x,0) = 0 \\
    \label{ch1:last}
    t_0&=0 .
  \end{align}

## Getting to know the Code

 As in the previous exercises you can control most of the settings through the ini-file `tutorial03.ini` or directly within the notebook. Get an overview of the configurable settings and run the code.

The program writes output with the extension `pvd`. This is one of several ways to write VTK output for the instationary case, c.f. the documentation of the `tutorial03`. The `pvd`-file can be visualized by ParaView and it consists of a collection of the corresponding `vtu`-files.
One big advantage of this approach is that the physical time can be printed out. This can be achieved by using the ``AnnotateTimeFilter`` in ParaView.

 - *the video below shows the solution for the nonlinear heat equation*

<video controls src="ex01nonLin.ogv" width="100%"/>

## Making Discretizations easily exchangable

**Step 1: Switching to the linear heat equation**
 For the rest of the exercise we want to consider the linear heat equation. Therefore the reaction term has to be set to $q =  0$. Recompile and rerun `exercise03.cc` and investigate the difference to the nonlinear reaction term.
 


 - *example video linear heat equation*

<video controls src="ex02lin.ogv" width="100%"/>

<video controls src="ex02diff.ogv" width="100%"/>

**Hint for the rest of the exercise:** For different runs of the simulation you can change the output filename in `tutorial03.ini`.

Since the initial problem was nonlinear, Newton's method is used to solve the discretized equations. For the linear case it is sufficient to use the class `StationaryLinearProblemSolver`. Search in the notebook for the lines starting with

```c++
using PDESOLVER = Dune::PDELab::Newton<IGO,LS,Z>;
PDESOLVER pdesolver(igo,z,ls);
```

and change to the `StationaryLinearProblemSolver`.

It has the same template arguments as the Newton solver. Give the instance of the class `StationaryLinearProblemSolver` also the name `pdesolver`. If you have problems with the construction of this solver consider for example the code in the driver of `tutorial00`. Compile and run again. The program reports the status of the solver. Get used to these different two outputs.

```c++  
//using PDESOLVER = Dune::PDELab::Newton<IGO,LS,Z>;
//PDESOLVER pdesolver(igo,z,ls);

using PDESOLVER = Dune::PDELab::StationaryLinearProblemSolver<IGO,LS,Z>;
PDESOLVER pdesolver(igo,ls,z,1e-10);
```

As a next step we want to use two spatial discretizations, i.e. $\mathcal{Q}_1$ and $\mathcal{Q}_2$ elements. The degree of the spatial discretization can be changed in the ini-file. Currently $\mathcal{Q}_1$ elements are used. Please change to $\mathcal{Q}_2$ elements and rerun the simulation.

- *the video shows a comparison between $\mathcal{Q}_1$ (left) and $\mathcal{Q}_2$ (right) elements*

<video controls src="ex02deg2.ogv" width="100%"/>

**Step 2: Arbitrary one-step schemes** We want to examine the numerical solution under three different time
  discretization schemes -- Implicit Euler, Crank-Nicolson and Fractional-Step-$\theta$. In order to change the time discretization scheme you will have to go to the<font color = 'red'> file </font> `driver.hh` and search for the line
```c++
    Dune::PDELab::Alexander2Parameter<RF> pmethod;
```

Change this to use the`Dune::PDELab::ImplicitEulerParameter<RF>`, compile and rerun the simulation. The program reports the progress of the time stepping and the method used. Convince yourself that you are using indeed the Implicit Euler. The other two time stepping methods can be applied similarly.

  **Note that** there is no special one step parameter class for Crank-Nicolson.  Crank-Nicolson is however the special case of the one step $\theta$ scheme with $\theta=0.5$.  So you can create a parameter
  object for Crank-Nicolson with `Dune::PDELab::OneStepThetaParameter<RF>(0.5)`.  A parameter object for the Fractional Step $\theta$ scheme can be created with `Dune::PDELab::FractionalStepParameter<RF>()`.

 - Implicit Euler method
 ```c++
 Dune::PDELab::ImplicitEulerParameter<RF> pmethod;
 ```
 - Crank-Nicolson method
 ```c++
Dune::PDELab::OneStepThetaParameter<RF> pmethod(0.5);
```
 - Fractional-Step-$\theta$
```c++
 Dune::PDELab::FractionalStepParameter<RF> pmethod;
```

**Step 3: Different Initial and Boundary Conditions** 
Consider the initial and boundary conditions
  (\ref{ch1:first})--(\ref{ch1:last}) modified as follows:
  \begin{align}
    \label{ch2:first}
    \Gamma_D&=\emptyset \\
    u_0(x) &= g(x, 0) = \prod_{i=0}^{d-1}\min\{1,\max\{0,\tilde{f}(x_i)\}\} \\
        & \qquad\qquad \tilde{f}(\xi):=0.5-8(|\xi-0.5|-0.25) \nonumber \\
    \label{ch2:last}
    j(x,t)&=0 .
  \end{align}
The initial condition given by $u_0$ models a block of constant 1 concentration in the middle, constant 0 concentration at the border and some linear decrease in between.

<img src = "initialBC.png" width="40%">

On a $16\times16$ or finer grid the initial values can be represented exactly by $\mathcal{Q}_1$ and $\mathcal{Q}_2$ finite elements. The exact solution will instantly become smooth and tend toward the mean over time.  A computed solution is only an approximation, and may show different behavior.  Most often it may
take a long time for the solution to become smooth and, depending on the time stepping scheme used, there are spikes oscillating from one time step to the next.

Please implement the initial and boundary conditions (\ref{ch2:first})--(\ref{ch2:last}). When implementing $u_0$ you may be tempted to use the function `abs()`. This is wrong, `abs()` is a function from the C-library to compute absolute values for *integers*. If a floating point value is plugged in, it is truncated to an integer. The correct way is to use `std::abs()` instead.

Compile and run your program. Remember that the grid needs $16\times16$ elements for the $\mathcal{Q}_1$/$\mathcal{Q}_2$ elements to resolve the initial condition exactly. What happens to the interpolated initial condition if you use a coarser mesh?

<video controls src="ex03BC.ogv" width="100%"/>

 - *initial condition for grid with $8\times8$ elements, using $\mathcal{Q}_1$ elements*
<img src = "initBC.png" width="70%">

**Step 4: Investigate Maximum Principle**

With these preparations done, it is now time to actually check how the different discretizations perform.  Run your program to produce some output that you can examine in ParaView. Change the settings in the ini-file to a $64\times 64$ grid and the time step size `<dt>`$\,=1/64=0.015625$. Run the simulation until `<tend>`$\,=4 \cdot\text{dt}$.
 When examining the solution in ParaView, apply the "Warp by Scalar" filter to get an image distorted into the third dimension according to the values of the solution.

After one time step, the solution computed by $\mathcal{Q}_1$ finite elements with Implicit Euler time stepping should be completely smooth. The same goes for $\mathcal{Q}_2$ with Implicit Euler.

With both Crank-Nicolson and Fractional Step $\theta$, both with $\mathcal{Q}_1$ and $\mathcal{Q}_2$ the solution should be quite non-smooth, i.e. there should be some edges visible. The Fractional Step $\theta$ scheme should be smoother than Crank-Nicolson.

Try to run the simulation with smaller time steps.  How small do you need to make the time steps to get smooth solutions with Fractional Step $\theta$?

Can you get the Crank-Nicolson scheme to produce smooth solutions as well?

## Time Dependent $g$ and $j$

Consider now the initial and boundary conditions (\ref{ch1:first})--(\ref{ch1:last}) time dependent:
\begin{align}
  \label{ch3:first}
  \Gamma_D &= \{ x\in\partial\Omega \mid x_0 = 0 \} \\
  g(x,t) &= t/10 \\
  u_0(x) &= g(x,0) \\
  \label{ch3:last}
  j(x,t) &= -(0.5 + \cos(t)/2) .
  \end{align}
Incorporating the time dependence into the functions $g$ and $j$ is easy even if they don't have the time variable as an argument. The problem parameter class possesses the member variable `t` and the member function
 ```c++
    void setTime (Number t_)
    {
      t = t_;
    }
 ```
by means of which the correct time is always available. Please implement the conditions (\ref{ch3:first})--(\ref{ch3:last}), compile and rerun the program. You might also want to increase the final time of the simulation. Examine the results in ParaView with the "Warp by Scalar" filter.


**A short note on time dependent $\Gamma_D$ and $\Gamma_N$:** In principle it is possible to implement time dependent $\Gamma_D$ and $\Gamma_N$ the same way as for $g$ and $j$. But for conforming spatial discretizations there is an important limiting assumption, namely that the type of boundary conditions do not change over a time step.

# Bibliographie

[1] Alexander, R. *Diagonally Implicit Runge-Kutta Methods for Stiff O. D. E.'s*. SIAM Journal on Numerical Analysis. Vol. 14. 1977.

[2] Bastian, P. *Lecture Notes on Scientific Computing with Partial Differential Equations*. 2014. http://conan.iwr.uni-heidelberg.de/teaching/numerik2_ss2014/num2.pdf. 

[3] Braess, D. *Finite Elemente*. Springer. 2003.

[4] Brenner, S. C. and Scott, L.R. *The mathematical theory of finite element methods*. Springer. 1994.

[5] Ciarlet, P. G. *The finite element method for elliptic problems*. SIAM. Classics in Applied Mathematics. 2002.

[6] Elman, H., Silvester, D. and Wathen, A. *Finite Elements and Fast Iterative Solvers*. Oxford University Press. 2005.

[7] Eriksson, K. ,Estep,D. Hansbo, P. and Johnson, C. *Computational Differential Equations*. Cambridge University Press. 1996. http://www.csc.kth.se/~jjan/private/cde.pdf.

[8] Ern, A. and Guermond, J.-L. *Theory and practice of finite element methods*. Springer. 2004.
<a id="ern"> </a>

 [9] Shu, C. W. and Osher, S. *Efficient implementation of essentially non-oscillatory shock-capturing schemes*. J. Comput. Phys. Vol. 77. 1988.
 <a id="ShuOsher88"> </a>

[10] Gottlieb, S., Ketcheson, D.I. and Shu, C.W. *Strong Stability Preserving Runge-Kutta and Multistep Time Discretizations*. World Scientific. 2011.
<a id="gottlieb11"> </a>

[11] Geuzaine, C. and Remacle, J.-F. *Gmsh: A 3-D finite element mesh generator with built-in pre- and post-processing facilities*. International Journal for Numerical Methods in Engineering. Vol. 79. 2009.

[12] Großmann, C. and Roos, H.-G. *Numerische Behandlung partieller Differentialgleichungen*. Teubner. 2006.

[13] Hackbusch, W. *Theorie und Numerik elliptischer Differentialgleichungen*. Teubner. 1986. http://www.mis.mpg.de/preprints/ln/lecturenote-2805.pdf.

[14] Meidner, D. and Richter, T. *A posteriori error estimation for the fractional step theta discretization of the incompressible Navier–Stokes equations*. Computer Methods in Applied Mechanics and Engineering. 2015.

[15] Rannacher, R. *Einführung in die Numerische Mathematik II (Numerik partieller Differentialgleichungen)*. 2006. http://numerik.iwr.uni-heidelberg.de/~lehre/notes. 

[16] Shu, C.W. *Total-variation-diminishing time discretizations*. SIAM J. Sci. Stat. Comput. Vol. 9. 1988.
<a id = "Shu"> </a>